In [ ]:
!git clone https://github.com/trzy/FasterRCNN

Cloning into 'FasterRCNN'...
remote: Enumerating objects: 1810, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1810 (delta 76), reused 140 (delta 76), pack-reused 1666 (from 1)
Receiving objects: 100% (1810/1810), 10.53 MiB | 13.18 MiB/s, done.
Resolving deltas: 100% (966/966), done.


In [ ]:
# !python -m venv tf2_venv
# !source tf2_venv/bin/activate
!pip install -r FasterRCNN/tf2/requirements.txt


# !pip install -r FasterRCNN/pytorch/requirements.txt

In [ ]:
!pip install roboflow

In [ ]:
#disini kita akan menggunakan dataset yang sama dengan kelompok pembanding
#dataset: https://universe.roboflow.com/mohamed-traore-2ekkp/face-detection-mik1i

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("mohamed-traore-2ekkp")
project = workspace.project("face-detection-mik1i")
version = project.version(3)
dataset = version.download("voc")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Face-Detection-3 in voc:: 100%|██████████| 4337/4337 [00:00<00:00, 11229.64it/s]


In [ ]:
!./FasterRCNN/download_models.sh

--2025-06-02 13:50:54--  http://trzy.org/files/fasterrcnn/vgg16_caffe.pth
Resolving trzy.org (trzy.org)... 72.46.129.2
Connecting to trzy.org (trzy.org)|72.46.129.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://trzy.org/files/fasterrcnn/vgg16_caffe.pth [following]
--2025-06-02 13:50:54--  https://trzy.org/files/fasterrcnn/vgg16_caffe.pth
Connecting to trzy.org (trzy.org)|72.46.129.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553433685 (528M)
Saving to: ‘vgg16_caffe.pth’

vgg16_caffe.pth     100%[===================>] 527.79M  82.2MB/s    in 7.0s    

2025-06-02 13:51:01 (75.9 MB/s) - ‘vgg16_caffe.pth’ saved [553433685/553433685]

--2025-06-02 13:51:01--  http://trzy.org/files/fasterrcnn/fasterrcnn_pytorch_vgg16.pth
Resolving trzy.org (trzy.org)... 72.46.129.2
Connecting to trzy.org (trzy.org)|72.46.129.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://trz

In [ ]:
!./FasterRCNN/download_dataset.sh

--2025-06-02 14:16:30--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  15.4MB/s    in 21s     

2025-06-02 14:16:52 (20.6 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]

--2025-06-02 14:16:52--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================

In [ ]:
!rm -rf /content/Face-Detection-3

In [ ]:
import os
import shutil
from glob import glob

# Paths
base_dir = "/content/Face-Detection-3"
ann_dir = os.path.join(base_dir, "Annotations")
img_dir = os.path.join(base_dir, "JPEGImages")
imagesets_main = os.path.join(base_dir, "ImageSets", "Main")
os.makedirs(ann_dir, exist_ok=True)
os.makedirs(img_dir, exist_ok=True)
os.makedirs(imagesets_main, exist_ok=True)

splits = ['train', 'valid', 'test']
for split in splits:
    split_dir = os.path.join(base_dir, split)
    img_files = glob(os.path.join(split_dir, "*.jpg"))
    print(f"[{split.upper()}] Found {len(img_files)} images in {split_dir}")

    txt_path = os.path.join(imagesets_main, f"{split}.txt")
    with open(txt_path, 'w') as f:
        for img_file in img_files:
            base = os.path.basename(img_file).replace(".jpg", "")
            f.write(base + "\n")
            # Move to VOC structure
            shutil.move(img_file, os.path.join(img_dir, os.path.basename(img_file)))
            xml_file = img_file.replace(".jpg", ".xml")
            if os.path.exists(xml_file):
                shutil.move(xml_file, os.path.join(ann_dir, os.path.basename(xml_file)))
            else:
                print(f"⚠️ Missing XML for {img_file}")

# Create trainval.txt (train + valid)
train_txt = os.path.join(imagesets_main, "train.txt")
val_txt = os.path.join(imagesets_main, "valid.txt")
trainval_txt = os.path.join(imagesets_main, "trainval.txt")

with open(trainval_txt, "w") as f_trainval:
    for fname in [train_txt, val_txt]:
        if os.path.exists(fname):
            with open(fname, "r") as f:
                lines = f.readlines()
                print(f"Adding {len(lines)} entries from {fname}")
                f_trainval.writelines(lines)
        else:
            print(f"❌ File missing: {fname}")

print("✅ Completed. Check contents of trainval.txt.")


[TRAIN] Found 1896 images in /content/Face-Detection-3/train
[VALID] Found 180 images in /content/Face-Detection-3/valid
[TEST] Found 90 images in /content/Face-Detection-3/test
Adding 1896 entries from /content/Face-Detection-3/ImageSets/Main/train.txt
Adding 180 entries from /content/Face-Detection-3/ImageSets/Main/valid.txt
✅ Completed. Check contents of trainval.txt.


In [ ]:
!python -m FasterRCNN.pytorch.FasterRCNN --train --epochs=10 --learning-rate=1e-3 --load-from="/content/vgg16_caffe.pth" #--dataset-dir="/content/Face-Detection-3"

Loaded initial VGG-16 layer weights from Caffe model '/content/vgg16_caffe.pth'
Error(s) in loading state_dict for FasterRCNNModel:
	Missing key(s) in state_dict: "_stage2_region_proposal_network._rpn_conv1.weight", "_stage2_region_proposal_network._rpn_conv1.bias", "_stage2_region_proposal_network._rpn_class.weight", "_stage2_region_proposal_network._rpn_class.bias", "_stage2_region_proposal_network._rpn_boxes.weight", "_stage2_region_proposal_network._rpn_boxes.bias", "_stage3_detector_network._pool_to_feature_vector._fc1.weight", "_stage3_detector_network._pool_to_feature_vector._fc1.bias", "_stage3_detector_network._pool_to_feature_vector._fc2.weight", "_stage3_detector_network._pool_to_feature_vector._fc2.bias", "_stage3_detector_network._classifier.weight", "_stage3_detector_network._classifier.bias", "_stage3_detector_network._regressor.weight", "_stage3_detector_network._regressor.bias". 
	Unexpected key(s) in state_dict: "_stage3_detector_network._fc1.weight", "_stage3_detecto

In [ ]:
import os
import xml.etree.ElementTree as ET

annotations_path = "/content/Face-Detection-3/Annotations"
class_names = set()

for xml_file in os.listdir(annotations_path):
    if not xml_file.endswith(".xml"):
        continue
    tree = ET.parse(os.path.join(annotations_path, xml_file))
    root = tree.getroot()
    for obj in root.findall("object"):
        name = obj.find("name").text.strip()
        class_names.add(name)

print("Classes found:", class_names)


Classes found: {'face'}


In [ ]:
!cd /content/Face-Detection-3/Annotations
!ls *.xml | sed 's/.xml$//' > /content/Face-Detection-3/ImageSets/Main/trainval.txt


ls: cannot access '*.xml': No such file or directory


In [ ]:
import os

# Paths
dataset_root = "/content/Face-Detection-3"
image_dir = os.path.join(dataset_root, "JPEGImages")
annot_dir = os.path.join(dataset_root, "Annotations")
split_file = os.path.join(dataset_root, "ImageSets/Main/trainval.txt")

# Check each entry in trainval.txt
with open(split_file, "r") as f:
    missing_images = []
    missing_annotations = []
    for line in f:
        name = line.strip()
        image_path = os.path.join(image_dir, f"{name}.jpg")
        annot_path = os.path.join(annot_dir, f"{name}.xml")

        if not os.path.isfile(image_path):
            missing_images.append(f"Missing image: {name}.jpg")

        if not os.path.isfile(annot_path):
            missing_annotations.append(f"Missing annotation: {name}.xml")

# Output results
if not missing_images and not missing_annotations:
    print("✅ All files are present.")
else:
    print("\n".join(missing_images + missing_annotations))


✅ All files are present.


In [ ]:
!python -m FasterRCNN.tf2.FasterRCNN \
  --train \
  --epochs=10 \
  --learning-rate=1e-3 \
  --load-from="/content/fasterrcnn_tf2.h5" \
  --dataset-dir="/content/Face-Detection-3"

2025-06-02 13:05:01.836909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748869501.861302    1486 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748869501.868205    1486 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 13:05:01.892031: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2025-06-02 13:05:06.385893: E external/local